In [12]:
Resume = [{'resume_url': 'https://jntavmoxtjnflnrsbulo.supabase.co/storage/v1/object/public/resumes/27/vaibhav.arora.102g@gmail.com-1759580936649-JD%20Software%20Engineer%20(Go)%20(1).pdf'}, {'resume_url': 'https://jntavmoxtjnflnrsbulo.supabase.co/storage/v1/object/public/resumes/27/aroravaibhav661@gmail.com-1759580948813-Vaibhav%20Arora%20Amazon.pdf'}]

In [13]:
import os
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [16]:
download_dir = "resumes"
os.makedirs(download_dir, exist_ok=True)

# ====================================
# 📥 STEP 2: Download & Combine PDFs
# ====================================
from langchain.docstore.document import Document

all_docs = []

for item in Resume:
    pdf_url = item['resume_url']
    file_name = os.path.join(download_dir, pdf_url.split("/")[-1])

    print(f"⬇️ Downloading {file_name} ...")
    response = requests.get(pdf_url)
    with open(file_name, "wb") as f:
        f.write(response.content)

    # Load PDF and combine all pages into one text
    loader = PyPDFLoader(file_name)
    pages = loader.load()
    full_text = "\n".join([p.page_content for p in pages])

    # Create one Document object per PDF
    doc = Document(
        page_content=full_text,
        metadata={"source": pdf_url, "filename": os.path.basename(file_name)}
    )
    all_docs.append(doc)

print(f"✅ Loaded {len(all_docs)} full documents")


⬇️ Downloading resumes\vaibhav.arora.102g@gmail.com-1759580936649-JD%20Software%20Engineer%20(Go)%20(1).pdf ...
⬇️ Downloading resumes\aroravaibhav661@gmail.com-1759580948813-Vaibhav%20Arora%20Amazon.pdf ...
✅ Loaded 2 full documents


In [21]:
# ====================================
# 🧬 STEP 3: Embed Full Docs & Store in FAISS
# ====================================
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(all_docs, embeddings)
vectorstore.save_local("faiss_resume_full_db")

print("✅ FAISS vector database saved at 'faiss_resume_full_db'")


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error